In [1]:
import sys
sys.path.append('../')
import os
import glob
import pandas as pd
import torch
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('agg' )
#%matplotlib inline
from torch.utils.data import Dataset, DataLoader

from lib.preprocessing import *
from lib.dataloading import *
from lib.loss_functions import *
from lib.evaluation import *
from torchvision import transforms
from torchvision.transforms import CenterCrop
import torchvision.models as models
from torch import optim, nn
from skimage.color import rgb2gray

In [2]:
transforms = get_transformer_norm()
dataset = SSIDataset(img_file= '/home/jimmy/Data/SSI/ssi.csv', transform=transforms['train'], output_resize = True)

In [11]:
gen_loss = GenLoss(loss_type = 'hinge')
dis_loss = DisLoss(loss_type = 'hinge')

In [9]:
batch_size = 4
dataloader = torch.utils.data.DataLoader(dataset, batch_size= batch_size)
dlabel = torch.FloatTensor(batch_size)

generator = CENet()
discriminator = LinearDiscriminator(n_input=3, n_classes=9)
epochs = 100
device = 'cuda:0'
generator = generator.to(device)
discriminator = discriminator.to(device)
dlabel = dlabel.to(device)
#criteriaBCE.to(device)

for epoch in range(epochs):
    print('Starting epoch {}/{}.'.format(epoch + 1, epochs))
    generator.train()
    discriminator.train()
        
    G_epoch_loss = 0         
    D_epoch_loss = 0

    for i, (imgs, centers, dcm_labels) in enumerate(dataloader):
        imgs, centers, dcm_labels = imgs.to(device), centers.to(device), dcm_labels.to(device)   
        
        pred_centers = generator(imgs)
        
        dis_fake = discriminator(pred_centers, dcm_labels)        
        # Generator Advasarial Loss                        
        
        lossAdv_Encoder = gen_loss(dis_fake)
                
        # Generator MSE Loss
        #lossMSE_Encoder = criteriaMSE(pred_centers, centers)
        
        #lossG = 0.99 lossAdv_Encoder + 0.01 lossMSE_Encoder
                                                                
        #G_epoch_loss += lossG.item()
        break
    break

Starting epoch 1/100.


In [10]:
lossAdv_Encoder

tensor(0.8788, device='cuda:0', grad_fn=<MeanBackward0>)

In [6]:
hinge_loss

tensor(-0.2378, device='cuda:0', grad_fn=<NegBackward>)

In [7]:
lossAdv_Encoder

tensor(2.0548, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

In [18]:
-torch.mean(output)

tensor(1.0459, device='cuda:0', grad_fn=<NegBackward>)

In [9]:
discriminator = LinearDiscriminator(n_input=3, n_classes=9)
discriminator.to(device)
dis_out = discriminator(centers, dcm_labels)

x1 torch.Size([4, 1])
tensor([[ 0.0335],
        [-0.2584],
        [ 0.0469],
        [ 0.1653]], device='cuda:0', grad_fn=<AddmmBackward>)
x2 torch.Size([4, 1])
tensor([[ 0.1404],
        [ 0.0909],
        [-0.0769],
        [ 0.1333]], device='cuda:0', grad_fn=<SumBackward1>)


In [11]:
dis_out

tensor([[ 0.1739],
        [-0.1675],
        [-0.0300],
        [ 0.2985]], device='cuda:0', grad_fn=<AddBackward0>)